In [ ]:
# 가상환경 실행 : .\.venv\Scripts\activate.ps1

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
#from langchain_community.llms import Ollama
#from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import initialize_agent, AgentType
#from langchain_ollama import OllamaEmbeddings
import os
from langgraph.prebuilt import ToolNode
from typing import Literal
#from langgraph.graph import END
from langgraph.graph import START, END
from langgraph.graph import MessagesState, StateGraph
from langchain_core.prompts import PromptTemplate


In [2]:
# model cell

embeddings = OllamaEmbeddings(
    model="bge-m3"
)

llm = ChatOllama(
    model="qwen3:4b"
)

C:\Users\worb1\AppData\Local\Temp\ipykernel_21804\3511307189.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [3]:
persist_directory = "./chroma_db"
collection_name = 'requirements_list'
if os.path.exists('./chroma_db') and len(os.listdir('./chroma_db')) > 0:
    #기존 벡터 DB 가 존재할 경우.
    print("Vector DB 존재. 불러오기 시작.")
    
    vector_store = Chroma(
        embedding_function=embeddings,
        collection_name = collection_name,
        persist_directory = persist_directory
    )
    print("Vector DB 불러오기 완료.")
else:
    print("Vector DB 부재. 생성 시작.")
    # 1. 문서 로드
    loader = TextLoader("./docs/RFP_requirements.md", encoding="utf-8")
    documents = loader.load()

    # 2. 문서 나누기
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(documents)
    print(f"Chroma DB에 {len(splits)}개의 문서를 임베딩하여 저장 완료.")

    # 3. 벡터 스토어 생성
    vector_store = Chroma.from_documents(
        documents=splits, 
        embedding=embeddings, 
        persist_directory=persist_directory,
        collection_name=collection_name    
    )
    print("Vector DB 생성 완료.")


Vector DB 존재. 불러오기 시작.
Vector DB 불러오기 완료.


In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

#query = "교육과정을 개설하는 기능은?"
#result = retriever.invoke(query)
#print(f"✅ 검색 결과: {result}")

✅ 검색 결과: [Document(id='ebd4e7e5-647a-46ad-bafa-b605a9e1a51c', metadata={'source': './docs/RFP_requirements.md'}, page_content='- **근거문서**:\n  - RFP III‑3 “상세 요구사항” SFR‑003 :contentReference[oaicite:2]{index=2}&#8203;:contentReference[oaicite:3]{index=3}\n\n- **비고**:\n  - 산출물: 요구사항정의서, ERD, API 설계서, 화면설계서  \n\n## SFR-004: 프로그램 및 과정 개설\n\n- **요구사항ID**: SFR-004\n- **유형**: 기능 요구사항\n- **대분류**: 구축 LMS (Learning Management System)\n- **중분류**: 프로그램 및 과정 개설\n- **소분류**:\n  - **교육과정 개설·관리 기능**\n    - 교육일자·교육차수·학습자 등을 고려한 과정 개설 관리\n    - 엑셀 업로드를 통한 대량 교육과정 등록\n    - 등록된 교육과정 리스트 조회\n    - 교육별 차수별 수강 대상자 조회·등록, 과정 게시판 및 수료처리 관리\n  - **교육과정 실시 정보 변경 관리 기능**\n    - 신청 결과 등록 및 최종 교육 프로그램 내역 확인\n    - 프로그램 기간 연장 시 변경 내용 자동 반영\n    - 교육과정 취소 기능\n  - **교육요금 관리 기능**\n    - 다양한 교육요금 정책 관리\n    - 교육료 일괄·일자별 관리\n\n- **요구사항명**: 프로그램 및 과정 개설\n- **설명**:  \n  교육과정 개설부터 관리, 변경·취소, 요금 정책 설정 및 학습자 대상자 등록까지 전 과정을 지원하는 기능을 제공해야 한다. :contentReference[oaicite:0]{index=0}&#8203;:contentReference[oaicite:1]{index=1}\n\n-

In [ ]:
from langchain_core.output_parsers import StrOutputParser
# 1. 소스코드 구현 내용 을 자연어로 변환 시키는 부분

code_interpreter_template = """
당신은 Code Review 전문가입니다.

지금부터 제공해주는 코드 파일들의 기능을 분석하세요.
분석 결과는 **아래의 출력 형식만** 따르세요.  
그 외의 어떤 설명, 영어 문장, `<think>` 등은 **절대 포함하지 마세요.**

information List : {information}

반드시 지켜야 할 규칙:
- 출력은 아래 형식만 사용합니다.
- 생각, 분석, 설명, 영어 문장은 절대 출력하지 마세요.
- `<think>`나 내부 분석 과정도 출력하지 마세요.
- 아래 형식처럼, 각 파일명과 기능을 한글로 요약해서 출력만 하세요.

출력 예시:
<output>
- A.java  
-- 사용자 등록 로직 구현.  
-- 사용자 삭제 로직 구현.  
-- 사용자 중복 여부 검사 로직 구현.  
</output>

이제 파일 목록을 분석한 후, 위 형식에 맞춰 출력만 하세요.
"""
code_interpreter_prompt = PromptTemplate.from_template(code_interpreter_template)
code_interpreter_result_chain = code_interpreter_prompt | llm | StrOutputParser()

code_information = [{
    "fileName" : "LoginService.java",
    "language": "java",
    "code" : """
        @Service
        public class LoginService {

            private final UserRepository userRepository;

            public LoginService(UserRepository userRepository) {
                this.userRepository = userRepository;
            }

            public boolean login(String email, String password) {
                return userRepository.findByEmail(email)
                        .map(user -> user.getPassword().equals(password))
                        .orElse(false);
            }
        }
    """}
    , 
    { 
    "fileName" : "mcp_server.py",
    "language": "python",
    "code" : """
        # 가상환경 실행 : .\.venv\Scripts\activate.ps1

        from mcp.server.fastmcp import FastMCP

        mcp = FastMCP(\"ppm\")

        @mcp.tool()
        def add(a: int, b: int) -> int:
            \"\"\"두 숫자를 더합니다.\"\"\"
            return a + b

        # dev : mcp dev ./fastmcp-server/mcp_server.py
        # prd : python ./fastmcp-server/mcp_server.py 
        if __name__ == \"__main__\":
            mcp.run(transport="stdio")
    """}]

result = code_interpreter_result_chain.invoke({"information": code_information})




# 2. 변환시킨 자연어를 retriever 에 넣는 부분



<>:55: SyntaxWarning: invalid escape sequence '\.'
<>:55: SyntaxWarning: invalid escape sequence '\.'
C:\Users\worb1\AppData\Local\Temp\ipykernel_21804\304708240.py:55: SyntaxWarning: invalid escape sequence '\.'
  # 가상환경 실행 : .\.venv\Scripts\activate.ps1


In [27]:
result

'<think>\nOkay, let\'s tackle this code review. The user provided two files: LoginService.java and mcp_server.py. I need to analyze each one and summarize their functionality in Korean according to the specified format.\n\nStarting with LoginService.java. It\'s a Java class annotated with @Service, so it\'s likely a Spring service. The class has a constructor that takes a UserRepository, which suggests it\'s dependent on a repository for user data. The main method is login, which takes email and password. It uses userRepository.findByEmail to find a user by email, then checks if the stored password matches the provided password. If the user is found and the password matches, it returns true; otherwise, false. So the functionality here is handling the login process by verifying user credentials.\n\nNext, mcp_server.py. It\'s a Python script that imports FastMCP from a module. There\'s a FastMCP instance initialized with "ppm". Then there\'s a tool decorator called add, which takes two i